In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
import cv2
from tqdm import tqdm

input_base = "/kaggle/input/plants-sorted-data"
output_base = "/kaggle/working/resized_data"
img_size = 224

os.makedirs(output_base, exist_ok=True)

for split in ["train", "valid"]:
    split_input = os.path.join(input_base, split)
    split_output = os.path.join(output_base, split)
    os.makedirs(split_output, exist_ok=True)

    for plant in os.listdir(split_input):
        plant_in = os.path.join(split_input, plant)
        plant_out = os.path.join(split_output, plant)
        os.makedirs(plant_out, exist_ok=True)

        for cls in os.listdir(plant_in):
            cls_in = os.path.join(plant_in, cls)
            cls_out = os.path.join(plant_out, cls)
            os.makedirs(cls_out, exist_ok=True)

            for img_name in tqdm(os.listdir(cls_in), desc=f"Processing {split}/{plant}/{cls}"):
                img_path = os.path.join(cls_in, img_name)

                try:
                    img = cv2.imread(img_path)
                    if img is None:
                        print(f"❌ Failed: {img_path}")
                        continue

                    img = cv2.resize(img, (img_size, img_size))
                    cv2.imwrite(os.path.join(cls_out, img_name), img)

                except Exception as e:
                    print(f"⚠️ Error with {img_path}: {e}")

print("✅ Resizing Complete! ✅")


In [ ]:
!pip install kaggle


In [ ]:
from PIL import Image
import os

root = "/kaggle/working/resized_data"
wrong = []

for split in ['train', 'valid']:
    for cls in os.listdir(os.path.join(root, split)):
        folder = os.path.join(root, split, cls)
        for img_file in os.listdir(folder):
            img_path = os.path.join(folder, img_file)
            try:
                img = Image.open(img_path)
                if img.size != (224, 224):
                    wrong.append(img_path)
            except:
                print("Corrupt:", img_path)

print("✅ Checking completed!")
if len(wrong) == 0:
    print("🎯 All images are correctly resized to 224x224!")
else:
    print("⚠ Incorrect size images found:", len(wrong))
    print("Examples:", wrong[:10])


In [ ]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


In [ ]:
import os
import shutil
from pathlib import Path

source_base = Path('/kaggle/input/plants-sorted-data')
target_base = Path('/kaggle/working/merged_data')

# ✅ Define final 6 classes
allowed_classes = [
    'Bacterial_spot',
    'Early_blight',
    'Late_blight',
    'Leaf_scorch',
    'Powdery_mildew',
    'Healthy'
]

# ✅ Make merged directories
for split in ['train', 'valid']:
    for cls in allowed_classes:
        os.makedirs(target_base / split / cls, exist_ok=True)

# ✅ Copy images belonging to allowed classes
for split in ['train', 'valid']:
    for root, dirs, files in os.walk(source_base / split):
        for file in files:
            src_path = Path(root) / file
            class_name = Path(root).name

            if class_name in allowed_classes:
                dst_path = target_base / split / class_name / file
                shutil.copy(src_path, dst_path)

print("✅ Merging completed successfully!")


In [ ]:
import os
from pathlib import Path

for split in ['train', 'valid']:
    print(f'\n📌 {split.upper()} SET CLASS COUNTS')
    for cls in allowed_classes:
        count = len(os.listdir(target_base / split / cls))
        print(f'{cls}: {count} images')


In [ ]:
import os
import shutil
from sklearn.model_selection import train_test_split

source_dir = "/kaggle/input/plants-sorted-data"
output_base = "/kaggle/working/final_dataset"

splits = ["train", "valid", "test"]
classes = ["Healthy", "Bacterial_spot", "Early_blight", "Late_blight", "Leaf_scorch", "Powdery_mildew"]

# ✅ Create folders
for split in splits:
    for cls in classes:
        os.makedirs(os.path.join(output_base, split, cls), exist_ok=True)

# ✅ Gather all images first (merge)
all_class_images = {cls: [] for cls in classes}

for plant in os.listdir(os.path.join(source_dir, "train")):
    plant_path = os.path.join(source_dir, "train", plant)

    if os.path.isdir(plant_path):
        for cls in os.listdir(plant_path):
            if cls in all_class_images:
                for img in os.listdir(os.path.join(plant_path, cls)):
                    all_class_images[cls].append(os.path.join(plant_path, cls, img))

for plant in os.listdir(os.path.join(source_dir, "valid")):
    plant_path = os.path.join(source_dir, "valid", plant)

    if os.path.isdir(plant_path):
        for cls in os.listdir(plant_path):
            if cls in all_class_images:
                for img in os.listdir(os.path.join(plant_path, cls)):
                    all_class_images[cls].append(os.path.join(plant_path, cls, img))

# ✅ Split into 70% train, 20% valid, 10% test
for cls, images in all_class_images.items():
    train_imgs, temp_imgs = train_test_split(images, test_size=0.30, random_state=42)
    valid_imgs, test_imgs = train_test_split(temp_imgs, test_size=1/3, random_state=42)

    for img in train_imgs:
        shutil.copy2(img, os.path.join(output_base, "train", cls, os.path.basename(img)))
    for img in valid_imgs:
        shutil.copy2(img, os.path.join(output_base, "valid", cls, os.path.basename(img)))
    for img in test_imgs:
        shutil.copy2(img, os.path.join(output_base, "test", cls, os.path.basename(img)))

print("✅ Successfully merged + resplit into 70% Train / 20% Valid / 10% Test")


In [ ]:
import os

base = "/kaggle/working/final_dataset"
for split in ["train", "valid", "test"]:
    print(f"\n📌 Checking {split} folders:")
    print(os.listdir(os.path.join(base, split)))


In [ ]:
import os

base = "/kaggle/working/final_dataset"
splits = ["train", "valid", "test"]

for split in splits:
    print(f"\n📌 {split.upper()} SET CLASS COUNTS")

    split_path = os.path.join(base, split)
    for cls in os.listdir(split_path):
        cls_path = os.path.join(split_path, cls)
        if os.path.isdir(cls_path):
            print(f"{cls}: {len(os.listdir(cls_path))} images")


In [ ]:
import matplotlib.pyplot as plt
import cv2
import random

base = "/kaggle/working/final_dataset/train"
classes = os.listdir(base)

for i in range(6):
    cls = random.choice(classes)
    img_name = random.choice(os.listdir(os.path.join(base, cls)))
    img_path = os.path.join(base, cls, img_name)
    
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    plt.subplot(2, 3, i+1)
    plt.imshow(img)
    plt.title(cls)
    plt.axis("off")

plt.show()


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_dir = "/kaggle/working/final_dataset/train"
valid_dir = "/kaggle/working/final_dataset/valid"
test_dir = "/kaggle/working/final_dataset/test"

img_size = (224, 224)
batch_size = 32

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True,
)

valid_test_datagen = ImageDataGenerator(rescale=1./255)

train_data = train_datagen.flow_from_directory(
    train_dir, target_size=img_size,
    batch_size=batch_size, class_mode='categorical'
)

valid_data = valid_test_datagen.flow_from_directory(
    valid_dir, target_size=img_size,
    batch_size=batch_size, class_mode='categorical'
)

test_data = valid_test_datagen.flow_from_directory(
    test_dir, target_size=img_size,
    batch_size=batch_size, class_mode='categorical',
    shuffle=False
)


In [ ]:
from tensorflow.keras.optimizers import Adam


In [ ]:
train_path = "/kaggle/working/sorted_data_merged_split/train"
valid_path = "/kaggle/working/sorted_data_merged_split/valid"
test_path  = "/kaggle/working/sorted_data_merged_split/test"

print(train_path, valid_path, test_path)


In [ ]:
train_path = "/kaggle/working/sorted_data_merged_split/train"
valid_path = "/kaggle/working/sorted_data_merged_split/valid"
test_path  = "/kaggle/working/sorted_data_merged_split/test"


In [ ]:
test_path = '/kaggle/working/sorted_data_merged_split/valid'


In [ ]:
import os

print(os.listdir('/kaggle/working'))


In [ ]:
import os

def find_image_dirs(root, exts=('.jpg','.jpeg','.png')):
    results = []
    for dirpath, dirnames, filenames in os.walk(root):
        count = sum(1 for f in filenames if f.lower().endswith(exts))
        if count>0:
            results.append((dirpath, count))
    return sorted(results, key=lambda x: -x[1])

# Search both working and input
print("Searching /kaggle/working ...")
working = find_image_dirs('/kaggle/working')
for p,c in working[:30]:
    print(p, c)

print("\nSearching /kaggle/input ...")
inp = find_image_dirs('/kaggle/input')
for p,c in inp[:30]:
    print(p, c)


In [ ]:
train_path = "/kaggle/working/final_dataset/train"
valid_path = "/kaggle/working/final_dataset/valid"
test_path  = "/kaggle/working/final_dataset/test"


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_data = ImageDataGenerator(rescale=1./255, 
                                horizontal_flip=True,
                                rotation_range=15,
                                zoom_range=0.2).flow_from_directory(
    train_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

valid_data = ImageDataGenerator(rescale=1./255).flow_from_directory(
    valid_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)


In [ ]:
test_data = ImageDataGenerator(rescale=1./255).flow_from_directory(
    test_path,
    target_size=(224,224),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)


In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam


In [ ]:
train_path = "/kaggle/working/final_dataset/train"
valid_path = "/kaggle/working/final_dataset/valid"
test_path  = "/kaggle/working/final_dataset/test"


In [ ]:
train_data = ImageDataGenerator(rescale=1./255,
                                rotation_range=20,
                                zoom_range=0.2,
                                width_shift_range=0.2,
                                height_shift_range=0.2,
                                horizontal_flip=True,
                                fill_mode='nearest') \
                                .flow_from_directory(
                                    train_path,
                                    target_size=(224,224),
                                    batch_size=32,
                                    class_mode='categorical'
                                )

valid_data = ImageDataGenerator(rescale=1./255).flow_from_directory(
                valid_path,
                target_size=(224,224),
                batch_size=32,
                class_mode='categorical'
            )

test_data = ImageDataGenerator(rescale=1./255).flow_from_directory(
                test_path,
                target_size=(224,224),
                batch_size=32,
                class_mode='categorical',
                shuffle=False
            )


In [ ]:
base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(224,224,3))
base_model.trainable = False  # Freeze first for transfer learning

x = GlobalAveragePooling2D()(base_model.output)
output = Dense(train_data.num_classes, activation='softmax')(x)

model = Model(base_model.input, output)

model.compile(optimizer=Adam(1e-4), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()


In [ ]:
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt


In [ ]:
history = model.fit(
    train_data,
    validation_data=valid_data,
    epochs=12,
    batch_size=32
) 


In [ ]:
from tensorflow.keras.optimizers import Adam

# ✅ Unfreeze last 50 layers of DenseNet for fine-tuning
for layer in base_model.layers[-50:]:
    layer.trainable = True

# ✅ Compile with lower learning rate for fine-tuning
model.compile(
    optimizer=Adam(learning_rate=1e-5),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# ✅ Train for 6 more epochs
history_ft = model.fit(
    train_data,
    validation_data=valid_data,
    epochs=6
)


In [ ]:
def plot_graphs(history, history_ft):

    acc = history.history['accuracy'] + history_ft.history['accuracy']
    val_acc = history.history['val_accuracy'] + history_ft.history['val_accuracy']

    loss = history.history['loss'] + history_ft.history['loss']
    val_loss = history.history['val_loss'] + history_ft.history['val_loss']

    epochs_range = range(len(acc))

    plt.figure(figsize=(12,5))

    plt.subplot(1,2,1)
    plt.plot(epochs_range, acc, label="Train Acc")
    plt.plot(epochs_range, val_acc, label="Val Acc")
    plt.title("Accuracy")
    plt.legend()

    plt.subplot(1,2,2)
    plt.plot(epochs_range, loss, label="Train Loss")
    plt.plot(epochs_range, val_loss, label="Val Loss")
    plt.title("Loss")
    plt.legend()

    plt.show()

# ✅ Call the function
plot_graphs(history, history_ft)


In [ ]:
def plot_graphs(history, history_fine):

    acc = history.history['accuracy'] + history_fine.history['accuracy']
    val_acc = history.history['val_accuracy'] + history_fine.history['val_accuracy']

    loss = history.history['loss'] + history_fine.history['loss']
    val_loss = history.history['val_loss'] + history_fine.history['val_loss']

    epochs_range = range(len(acc))

    plt.figure(figsize=(12,5))

    plt.subplot(1,2,1)
    plt.plot(epochs_range, acc, label="Train Acc")
    plt.plot(epochs_range, val_acc, label="Val Acc")
    plt.title("Accuracy")
    plt.legend()

    plt.subplot(1,2,2)
    plt.plot(epochs_range, loss, label="Train Loss")
    plt.plot(epochs_range, val_loss, label="Val Loss")
    plt.title("Loss")
    plt.legend()

    plt.show()

plot_graphs(history, history_fine)


In [ ]:
val_loss, val_acc = model.evaluate(valid_data)
print("✅ Validation Accuracy:", round(val_acc,4))
print("✅ Validation Loss:", round(val_loss,4))


In [ ]:
y_pred = model.predict(test_data)
y_pred_labels = np.argmax(y_pred, axis=1)
y_true = test_data.classes
labels = list(test_data.class_indices.keys())

print("\n📌 Classification Report:")
print(classification_report(y_true, y_pred_labels, target_names=labels))

cm = confusion_matrix(y_true, y_pred_labels)
plt.figure(figsize=(7,6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=labels, yticklabels=labels)
plt.title("Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.show()


In [ ]:
model.save("/kaggle/working/plant_disease_densenet_final.h5")


In [ ]:
import os
os.listdir("/kaggle/working")


In [ ]:
from tensorflow.keras.models import load_model

model = load_model("/kaggle/working/plant_disease_densenet_final.h5")
model.summary()


In [ ]:
test_loss, test_acc = model.evaluate(test_data)
print(f"✅ Test Accuracy: {test_acc:.4f}")
print(f"✅ Test Loss: {test_loss:.4f}")


In [ ]:
from tensorflow.keras.preprocessing import image
import numpy as np

img_path = "/kaggle/input/your_image.jpg"
img = image.load_img(img_path, target_size=(224, 224))
img_array = np.expand_dims(image.img_to_array(img)/255.0, axis=0)

pred = model.predict(img_array)
class_names = list(test_data.class_indices.keys())

print("Predicted Class:", class_names[np.argmax(pred)])
